# project description

经过几个世纪的强烈捕鲸，恢复鲸鱼种群仍然很难适应变暖的海洋，并且每天都在努力与工业捕鱼业争夺食物。

为了帮助鲸鱼保护工作，科学家们使用照片监控系统来监测海洋活动。他们使用鲸鱼尾巴的形状和镜头中的独特标记来识别他们正在分析的鲸鱼种类，并精心记录鲸鱼荚的动态和运动。在过去的40年里，大部分工作都由个别科学家手工完成，留下了大量未开发和未充分利用的数据。

在本次比赛中，您面临的挑战是建立一种算法来识别图像中的个体鲸鱼。您将分析Happywhale的数据库，其中包含来自研究机构和公共贡献者的超过25,000张图像。通过贡献，您将有助于为全球海洋哺乳动物种群动态开辟丰富的理解领域。

请注意，此竞赛在本质上与[此竞赛](https://www.kaggle.com/c/whale-categorization-playground)类似，具有扩展和更新的数据集。

我们要感谢[Happywhale](https://happywhale.com/)提供这些数据和问题。 Happywhale是一个平台，它使用图像处理算法让任何人提交他们的鲸鱼照片并自动识别。

# evaluation

Submissions are evaluated according to the Mean Average Precision $@ 5 (MAP@5)$:

$$MAP@5=\frac{1}{U}\sum_{u=1}^{U}\sum_{k=1}^{min(n,5)}P(k)$$

where `U` is the number of images, `P(k)` is the precision at cutoff `k`, and `n` is the number predictions per image.


# data description

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

看一下有哪些文件撒

In [ ]:
! ls ../input/

train.csv: 训练数据的一些数据

train: 训练图片

test: 测试图片

sample_submission.csv: 提交文件的一个样例，可以从这里拿id然后构造要提交的文件

## 首先看一看 `train.csv`

In [ ]:
train_org = pd.read_csv("../input/train.csv")
print("train shape:", train_org.shape)
train_org.head(5)

其实就是一个图片和id对应的表，我们在 `test` 中需要预测的数据就是这里的 `Id`

In [ ]:
pd.DataFrame(train_org.Id.value_counts().describe([0.25, 0.40, 0.75, 0.8, 0.9, 0.95, 0.99, 0.999])).T

1. 可以看到，有近 **`40%`** 的鲸鱼，都只有一张照片，那么我们肯定需要数据增强了；
2. 最多的鲸鱼有 **`9664`** 张图片，我们是否需要减少一点这样的图片？
3. 数据的整体分布，我们可以看看下图。

In [ ]:
_id_counts = train_org.Id.value_counts()
_id_counts[_id_counts <= 20].hist(bins=21)
_ = plt.xlim(0, 20)

过滤掉了大于20张图片的样例，因为这些都太少了。

可以发现，大多数的图片都少于5张。接下来怎么办再商量撒。

## 再来看一看 `train`

In [ ]:
import os

from PIL import Image

In [ ]:
path_train = "../input/train/"
filenames = os.listdir(path_train)
filenames[0]

In [ ]:
with Image.open(os.path.join(path_train, filenames[0]), 'r') as img:
    print("filename:", img.filename)
    print("image shape:", np.array(img).shape)
    plt.imshow(img)

In [ ]:
with Image.open(os.path.join(path_train, "00d641885.jpg"), 'r') as img:
    print("filename:", img.filename)
    print("image shape:", np.array(img).shape)
    plt.imshow(img)

都是3通道图片，但是不都一样大，有些还不是彩色的。

- 是不是需要预处理一下？怎么预处理？

# submission

In [ ]:
submission = pd.read_csv("../input/sample_submission.csv")
print("test shape:", submission.shape)
submission.head()

测试数据中有 **`7960`** 个样本，需要我们每个样本最多预测5个类别，每个类别id用**空格**隔开。metric是啥，我们之后再说。

好了，接下来的，之后讨论。